# ORM_2

SQLAlchemy Object Relational Mapper는 유저가 정의한 DB에 대해서 파이썬으로 표현할 수 있도록 해준다.
DB에 대한 어떠한 변화가 투명하게 동기화시켜주는 시스템을 포함하고 있다.

SQL문과 달리 high level로 추상적인 패턴이다.

In [1]:
from sqlalchemy import create_engine

In [2]:
from sqlalchemy.ext.declarative import declarative_base #비어있는 base를 만듬

In [3]:
engine = create_engine("sqlite:///mydb.db", echo=True) #이미 만들어놓은 db파일 불러옴
# ///가 3개인 이유는 Host이기 때문...
# echo 플래그는 SQLAlchemy의 logging의 세팅 여부이다.
# engine을 한번 만들면 직접적으로 engine을 사용하진 않지만, ORM에 의해서 쓰일 것임.

In [4]:
from sqlalchemy.ext.automap import automap_base 
#자동으로 이미 생성되어 있는 db에서 schema를 불러오는 역할을 한다.

In [5]:
Base = automap_base() #자동으로 DB 불러옴
#Base = declarative_base() #비어있는 base 만듬

In [6]:
Base.prepare(engine, reflect = True) # reflect = true -> flag에 따라 DB 불러옴.

2018-10-11 15:21:03,092 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-11 15:21:03,115 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,119 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-11 15:21:03,123 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,127 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-10-11 15:21:03,130 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,135 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-10-11 15:21:03,137 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,142 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-10-11 15:21:03,144 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,149 INFO sqlalchemy.engine.base.

2018-10-11 15:21:03,335 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("addresses")
2018-10-11 15:21:03,338 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,342 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("addresses")
2018-10-11 15:21:03,346 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:21:03,349 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'addresses' AND type = 'table'
2018-10-11 15:21:03,351 INFO sqlalchemy.engine.base.Engine ()


In [7]:
from sqlalchemy.orm import sessionmaker

In [8]:
Session = sessionmaker(bind = engine) # engine binding한 세션

In [9]:
session = Session()  #세션 생성 -> DB를 수정.변경 할 수 있도록 해줌.

In [10]:
artist = Base.classes.Artist # automap_base()로 불러온 db의 Artist 테이블

In [11]:
artist.name

In [12]:
result = session.query(artist) # 실행 했는데 바로 출력이 안되네??

In [13]:
type(result) # Query 클래스 객체이다.

sqlalchemy.orm.query.Query

In [14]:
for row in result:
    print(row)
#바로 출력이 안되는 이유는 instance로 갖고왔지만 우리가 못보는 것일뿐

2018-10-11 15:21:11,991 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-11 15:21:11,998 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist"
2018-10-11 15:21:12,004 INFO sqlalchemy.engine.base.Engine ()


In [15]:
for row in result:
    print(row.id, row.name) #그래서 클래스 형식으로 값을 접근해서 볼 수 있다.

2018-10-11 15:21:12,856 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist"
2018-10-11 15:21:12,862 INFO sqlalchemy.engine.base.Engine ()
1 Led Zepplin
2 AC/DC


In [44]:
from sqlalchemy.orm import relationship
from sqlalchemy import Column, Integer, String, ForeignKey

In [45]:
engine = create_engine("sqlite:///test_relationship.db", echo=True)

In [46]:
Base = declarative_base()
#Base = declarative_base() #비어있는 base 만듬

In [47]:
class Artist(Base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    album = relationship("Album", back_populates = "artist")
    
    def __repr__(self):
        return "<T'artist(name='%s')>" % (self.name)
    
class Album(Base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    artist = relationship("Artist", back_populates = "album")

class Genre(Base):
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(Base):
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))
    
# relationship은  DB의 JOIN을 할 필요 없이, JOIN문 처럼 relationship을 형성할 수 있도록 해준다.

In [48]:
Base.metadata.create_all(engine) 
# metadata 생성 -> engine에 데이터베이스를 연결할 수 있도록 해준다. 

2018-10-11 15:34:40,592 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-11 15:34:40,597 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,605 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-11 15:34:40,610 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,616 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-10-11 15:34:40,620 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,627 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-10-11 15:34:40,630 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,636 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-10-11 15:34:40,640 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,643 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-10-11 15:34:40,645 INFO sqlalchemy.engine.base.Engine ()
2018-10-11 15:34:40,649 INFO sqlalchemy.engin

In [49]:
artist1 = Artist(name = 'Led zeppelin') 
artist2 = Artist(name = 'AC/DC')

# Artist 테이블 객체 생성

In [50]:
artist1.album = [Album(title="IV"), Album(title = "Who Made Who")]
# artist1의 album attribute에 넣는다.

In [51]:
Session = sessionmaker(bind = engine) # engine binding한 세션

In [52]:
session = Session()  #세션 생성 -> DB를 수정.변경 할 수 있도록 해줌. 

In [53]:
session.add(artist1) # session에 위에서 만든 artist 객체 추가

In [54]:
session

In [55]:
for row in session.query(Artist).all(): # 세션 내에서의 데이터를 query를 통해 도출
    print(row.name, row.album)

2018-10-11 15:34:53,198 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-11 15:34:53,206 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-10-11 15:34:53,211 INFO sqlalchemy.engine.base.Engine ('Led zeppelin',)
2018-10-11 15:34:53,223 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-10-11 15:34:53,227 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2018-10-11 15:34:53,232 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-10-11 15:34:53,235 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 1)
2018-10-11 15:34:53,240 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist
2018-10-11 15:34:53,242 INFO sqlalchemy.engine.base.Engine ()
Led zeppelin [<__main__.Album object at 0x0000029F77AB79E8>, <__main__.Album object at 0x0000029F7C053978>]


In [56]:
session.commit() # 위에서 session에 추가했던 데이터를 commit하면 실제 DB에 update해준다.

2018-10-11 15:35:33,023 INFO sqlalchemy.engine.base.Engine COMMIT
